# CSC416 - DigitalImage - HW#4 - Rinty Chowdhury

In [11]:
import numpy as np
import math

## Co-occurrence matrix

In [12]:
def co_occurrence_matrix(image):
    result = np.zeros((256, 256), dtype=np.uint8)
    for x in np.arange(0, image.shape[0] - 1):
        for y in np.arange(0, image.shape[1]):
            x_value = image.item(x + 1, y)
            y_value = image.item(x, y)
            matrix_value = result.item(x_value, y_value)
            result.itemset((x_value, y_value), matrix_value + 1)
    return result

In [13]:
cktboard = open('image_file/cktboard.raw', 'rb')
width = 365
height = 120
image = np.fromfile(cktboard, dtype=np.uint8, count = height * width)
image_array = image.reshape((height, width))
cktboard.close()
matrix = co_occurrence_matrix(image_array)
import cv2
eualize_matrix = cv2.equalizeHist(matrix)
eualize_matrix.tofile('output_images/cktboard_texture.raw')

## Similarity measure – correlation coefficient

In [14]:
def normalization(image):
    w, h = image.shape
    minimum = np.amin(image)
    if minimum < 0:
        image = image + abs(minimum)
    maximum = np.amax(image)
    result = np.zeros(image.shape, 'd')
    for x in np.arange(0, w):
        for y in np.arange(0, h):
            result.itemset((x, y), float(image.item(x, y)) / maximum)
    return result

import timeit as tit
def correlation_coefficient(image, template):
    t = tit.Timer()
    result = np.zeros(image.shape)
    image_width, image_height = image.shape
    template_width, template_height = template.shape
    template_mean = np.mean(template)
    start_time = t.timer()
    for i in np.arange(0, image_width):
        for j in np.arange(0, image_height):
            if i - template_width / 2 <= 0:
                left = 0
            elif image_width - i < template_width:
                left = image_width - template_width
            else:
                left = i
            right = left + template_width
            if j - template_height / 2 <= 0:
                top = 0
            elif image_height - j < template_height:
                top = image_height - template_height
            else:
                top = j
            bottom = top + template_height
            sub = image[left:right, top:bottom]
            sub_mean = np.mean(sub)
            temp = (sub - sub_mean) * (template - template_mean)
            s1 = temp.sum()
            temp = (sub - sub_mean) * (sub - sub_mean)
            s2 = temp.sum()
            temp = (template - template_mean) * (template - template_mean)
            s3 = temp.sum()
            denom = s2 * s3
            if denom == 0:
                temp = 0
            else:
                temp = s1 / math.sqrt(denom)
            result.itemset((i, j), temp)
    end_time = t.timer()
    print("Correlation coefficient: ", end_time - start_time)
    return result

In [15]:
flowers = open('image_file/flowers.raw', 'rb')
width = 400
height = 300
image = np.fromfile(flowers, dtype=np.uint8, count = height * width)
image_array = image.reshape((height, width))
flowers.close()
flowers_template = open('image_file/flowers-template.raw', 'rb')
width = 42
height = 45
image_template = np.fromfile(flowers_template, dtype=np.uint8, count = height * width)
image_template_array = image_template.reshape((height, width))
flowers_template.close()
correlation_coefficient = correlation_coefficient(image_array, image_template_array)
normalized_correlation_coefficient = normalization(correlation_coefficient) * 255
normalized_correlation_coefficient = normalized_correlation_coefficient.astype(np.uint8)
normalized_correlation_coefficient.tofile('output_images/flowers_correlation_coefficient.raw')

Correlation coefficient:  8.674546099999816


## Image downsampling and upsampling

In [16]:
def downsampling(image):
    even_row = np.delete(image, list(range(0, image.shape[0], 2)), axis=0)
    even_column = np.delete(even_row, list(range(0, even_row.shape[1], 2)), axis=1)
    return even_column

def upsampling(image):
    result = np.zeros((512, 512), dtype=np.uint8)
    for x in np.arange(0, image.shape[0]):
        for y in np.arange(0, image.shape[1]):
            result.itemset((x * 2, y * 2), image.item(x, y))
            result.itemset((x * 2 + 1, y * 2), image.item(x, y))
            result.itemset((x * 2, y * 2 + 1), image.item(x, y))
            result.itemset((x * 2 + 1, y * 2 + 1), image.item(x, y))
    return result

In [17]:
lena = open('image_file/lena.raw', 'rb')
width = 512
height = 512
image = np.fromfile(lena, dtype=np.uint8, count = height * width)
image_array = image.reshape((height, width))
lena.close()
down_sample = downsampling(image_array)
up_sample = upsampling(down_sample)
up_sample.tofile('output_images/lena_resampling.raw')